# Initial Table Setup

In [ ]:
from os import path
import pandas
from datetime import datetime
from config import db
from utils import snake_case

FOLDER = 'data'
RECORDS = 'FALL_KH101.xlsx'
LAB = 'Labor 4Q_2015_KH101.xlsx'
ICD = 'ICD_KH101.xlsx'
OPS = 'OPS_KH101.xlsx'
TARIFTABELLE = 'Tariftabelle_Rohdaten.xlsx'

dates = {
    'aufnahmedatum': '%Y%m%d%H%M',
    'entlassungsdatum': '%Y%m%d%H%M',
    'behandlungsbeginn_vorstationaer': '%Y%m%d',
    'behandlungsende_nachstationaer': '%Y%m%d',
    'admd': '%d.%m.%Y',
    'freeddate': '%Y-%m-%d %H:%M:%S',
    'ops_datum': '%Y%m%d%H%M',
    'aufn_datum': '%m/%d/%Y', # does seemingly not work
    'datf': '%d.%m.%Y',
    'datt': '%d.%m.%Y',
    'crd': '%d.%m.%Y',
    'chd': '%d.%m.%Y',
}

bools = {
  'belegoperateur': 'J/N',
  'beleganaesthesist': 'J/N',
  'beleghebamme': 'J/N',
}

colmap = {
    'alter': 'age',
    '5_ops': 'five_ops',
}

## Create Tables
Create tables from sql files. Drops all tables before recreating them, so watchout for data loss!


In [ ]:
# How to get formated column names:
# from utils import snake_case
# records = pandas.read_excel(path.join(FOLDER, ICD), header=1)
# cols = (snake_case(record) for record in records)
# cols = (colmap[col] if col in colmap else col for col in cols)

sql_files = [
    'sql/records.sql',
    'sql/lab.sql',
    'sql/icd.sql',
    'sql/ops.sql',
    'sql/tarifs.sql',
]

with db:
    for sql_file_name in sql_files:
        with open(sql_file_name, 'r') as sql_file:
            sql = sql_file.read()
        table_name = path.splitext(path.basename(sql_file_name))[0]
        try:
            db.execute('DROP TABLE IF EXISTS {};'.format(table_name))
        except sqlite3.OperationalError as error:
            print('SQLite couldn\'t drop table %s \n %s' % (table_name, error))
            exit(1)
        try:
            db.execute(sql)
        except sqlite3.OperationalError as error:
            print('SQLite Query caused error.\n\n%s\n\n%s' % (sql, error))
            exit(1)

In [ ]:
def insert_many(table, records):
    def insert(table, cols, rows):
        wild = ', '.join('?' * len(cols))
        colstr = ', '.join(cols)
        with db:
            query = 'INSERT INTO {} ({}) VALUES ({})'.format(table, colstr, wild)
            return db.executemany(query, rows)
        
    c = db.execute('PRAGMA table_info({})'.format(table))
    cols = [tup[1] for tup in c.fetchall()]
    if '%s_id' % table in cols: cols.remove('%s_id' % table)
    csvcols = [snake_case(csvcol) for csvcol in records]
    csvcols = [colmap[c] if c in colmap else c for c in csvcols]
    colkeys = [csvcols.index(c) for c in cols]

    rows = []
    for rowidx, record in records.iterrows():
        values = list(record[colkeys])
        for idx, val in enumerate(values):
            if cols[idx] in dates.keys():
                try:
                    val = int(val) if isinstance(val, float) else val
                    values[idx] = datetime.strptime(str(val), dates[cols[idx]])
                except ValueError:
                    values[idx] = None
            if cols[idx] in bools.keys():
                values[idx] = bools[cols[idx]].split('/').index(val) == 0
            if val != val: # NaN
                values[idx] = None
        rows.append(values)
        
        # Intermediate insert
        if len(rows) >= 1000:
            print('inserted {} of {} rows...'.format(rowidx+1, len(records.index)))
            insert(table, cols, rows)
            rows = []

    # Final insert
    insert(table, cols, rows)

## Insert records data
Reads the `RECORDS` file defined above and inserts all its records into the `records` table according to the inthere defined columns. Fails if any of the records are already in there (unique constraints).

In [ ]:
records = pandas.read_excel(path.join(FOLDER, RECORDS), header=2)
insert_many('records', records)

## Insert lab data
Reading the excel sheet will take a while. Needs some better way for bigger datasets in the future.

In [ ]:
records = pandas.read_excel(path.join(FOLDER, LAB), header=0)

In [ ]:
insert_many('lab', records)

## Insert ICD data

In [ ]:
records = pandas.read_excel(path.join(FOLDER, ICD), header=1)
insert_many('icd', records)

## Insert OPS data

In [ ]:
records = pandas.read_excel(path.join(FOLDER, OPS), header=0)
insert_many('ops', records)

## Insert tarifs

In [ ]:
records = pandas.read_excel(path.join(FOLDER, TARIFTABELLE), header=0)
insert_many('tarifs', records)